# temporal network

> utilities holding temporal network info

In [ ]:
#| default_exp tnet

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import math
import numpy as np
from moraine.utils_ import ngjit

In [ ]:
#| export
def _imagepair_from_bandwidth(nimages,bandwidth=None):
    if bandwidth is None: bandwidth = nimages
    assert nimages >= bandwidth
    ref, sec = np.triu_indices(nimages, 1)
    idx = np.where((sec-ref)<=bandwidth)
    return np.stack((ref[idx], sec[idx]),axis=-1).astype(np.int32)

In [ ]:
#| export
@ngjit
def _are_edges_sorted(edges):
    num_edges = edges.shape[0]
    for i in range(num_edges - 1):
        if edges[i, 0] > edges[i + 1, 0] or (edges[i, 0] == edges[i + 1, 0] and edges[i, 1] > edges[i + 1, 1]):
            return False
    return True

In [ ]:
#| export
@ngjit
def _are_edges_directed(edges):
    '''directed = reference index smaller than secondary index'''
    num_edges = edges.shape[0]
    for i in range(num_edges):
        if edges[i, 0] >= edges[i, 1]:
            return False
    return True

In [ ]:
#| export
@ngjit
def _are_edges_connected(edges):
    num_edges = edges.shape[0]

    current_ref = -1
    for i in range(num_edges):
        if edges[i,0] == current_ref:
            continue
        else:
            if edges[i,0] != edges[i,1]-1:
                return False
            current_ref = edges[i,0]
    return True

In [ ]:
#| export
class TempNet(object):
    def __init__(self,image_pairs,check_if_valid=True):
        if check_if_valid:
            are_edges_sorted = _are_edges_sorted(image_pairs)
            if not are_edges_sorted:
                raise ValueError('input image pairs are not sorted.')
            are_edges_directed = _are_edges_directed(image_pairs)
            if not are_edges_directed:
                raise ValueError('input image pairs are not directed (reference index larger than or equal to secondary index).')
            are_edges_connected = _are_edges_connected(image_pairs)
            if not are_edges_connected:
                raise ValueError('input image pairs are not connected.')
        self._image_pairs = image_pairs.astype(np.int32)
        self._n_images = self._image_pairs[-1,-1]+1
        self._image_pairs_1d = self._image_pairs[:,0]*self._n_images+self._image_pairs[:,1]

    @classmethod
    def from_bandwidth(cls, nimages, bandwidth=None):
        image_pairs = _imagepair_from_bandwidth(nimages,bandwidth)
        return cls(image_pairs,check_if_valid=False)

    @property
    def image_pairs(self):
        return self._image_pairs

    def image_pairs_idx(
        self,
        image_pairs=None,
        ref=None,
        sec=None,
    ):
        single_input = False
        if image_pairs is not None:
            ref = image_pairs[:,0]
            sec = image_pairs[:,1]
        else:
            if isinstance(ref,int):
                assert isinstance(sec,int)
                single_input = True
                ref = np.array([ref,]); sec = np.array([sec,])
            else:
                ref = np.array(ref); sec = np.array(sec)
            assert len(ref) == len(sec)
        
        query_1d = ref*self._n_images+sec
        mask = np.isin(query_1d,self._image_pairs_1d,assume_unique=True,invert=True)
        result = np.searchsorted(self._image_pairs_1d,query_1d)
        result[mask] = -1
        if single_input:
            result = result[0]
        return result

    def save(self,path:str, # zarr path
            ):
        '''Save the TempNet.'''
        tempnet_zarr = zarr.open(path,'w',shape=self._image_pairs.shape,dtype=self._image_pairs.dtype)
        tempnet_zarr[:] = self._image_pairs[:]

    @classmethod
    def load(cls, zarr_path:str, # zarr path
            ):
        '''classmethod to load the saved HilbertRtree.'''
        tempnet_zarr = zarr.open(path,'r')
        return cls(tempnet_zarr[:], check_if_valid=False)

Usage:

Create a temporal network by specifing the number of images and the temporal bandwidth (the maximum number of images to be paired with one image):

In [ ]:
tnet = TempNet.from_bandwidth(5,2)

In [ ]:
tnet.image_pairs

array([[0, 1],
       [0, 2],
       [1, 2],
       [1, 3],
       [2, 3],
       [2, 4],
       [3, 4]], dtype=int32)

Or, you can specify your own image pairs:

In [ ]:
tnet = TempNet(np.stack(([0,1,2,3],[1,2,3,4]),axis=-1))
tnet.image_pairs

array([[0, 1],
       [1, 2],
       [2, 3],
       [3, 4]], dtype=int32)

In [ ]:
tnet.image_pairs_idx(ref=1,sec=2)

1

In [ ]:
tnet.image_pairs_idx(ref=[1,2],sec=[2,3])

array([1, 2])

In [ ]:
#| export
def nimage_from_npair(npair):
    '''infer number of images from number of pairs assuming all pairs are selected.'''
    possible_nimage = math.ceil(math.sqrt(2*npair))
    for i in [0,-1,1,-2,2]:
        possible_nimage = possible_nimage+i
        if possible_nimage*(possible_nimage-1) == 2*npair:
            return possible_nimage
    raise ValueError('input is not correct value of number of all pairs')

In [ ]:
nimage_from_npair(6)

4

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()